In [1]:
import torchvision.models as models

import timm
import copy

from HC701PROJECT.hc701fed.model.baseline import Baseline

In [2]:
model1 = Baseline('resnet50', pretrained=False)
model2 = copy.deepcopy(model1)
model3 = timm.create_model('resnet50', pretrained=False, num_classes=5)

In [3]:
import torch
model3.load_state_dict(torch.load('/home/xiangjianhou/hc701-fed/checkpoint_simclr/model_20230416195313_199.pth'), strict=False)

RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([128, 2048]) from checkpoint, the shape in current model is torch.Size([5, 2048]).
	size mismatch for fc.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([5]).

In [5]:
import torch
# model2.load_state_dict(torch.load('/home/xiangjianhou/hc701-fed/checkpoint_simclr/model_20230416032207_199.pth'),strict=False)
values_pretrained = torch.load('/home/xiangjianhou/hc701-fed/checkpoint_simclr/model_20230416195313_199.pth')
for p1,p2,k1,k2 in zip(model3.state_dict().values(),values_pretrained.values(),model3.state_dict().keys(),values_pretrained.keys()):
    print(p1.shape,p2.shape,k1,k2)

torch.Size([64, 3, 7, 7]) torch.Size([64, 3, 7, 7]) conv1.weight conv1.weight
torch.Size([64]) torch.Size([64]) bn1.weight bn1.weight
torch.Size([64]) torch.Size([64]) bn1.bias bn1.bias
torch.Size([64]) torch.Size([64]) bn1.running_mean bn1.running_mean
torch.Size([64]) torch.Size([64]) bn1.running_var bn1.running_var
torch.Size([]) torch.Size([]) bn1.num_batches_tracked bn1.num_batches_tracked
torch.Size([64, 64, 1, 1]) torch.Size([64, 64, 1, 1]) layer1.0.conv1.weight layer1.0.conv1.weight
torch.Size([64]) torch.Size([64]) layer1.0.bn1.weight layer1.0.bn1.weight
torch.Size([64]) torch.Size([64]) layer1.0.bn1.bias layer1.0.bn1.bias
torch.Size([64]) torch.Size([64]) layer1.0.bn1.running_mean layer1.0.bn1.running_mean
torch.Size([64]) torch.Size([64]) layer1.0.bn1.running_var layer1.0.bn1.running_var
torch.Size([]) torch.Size([]) layer1.0.bn1.num_batches_tracked layer1.0.bn1.num_batches_tracked
torch.Size([64, 64, 3, 3]) torch.Size([64, 64, 3, 3]) layer1.0.conv2.weight layer1.0.conv2.wei

In [3]:
for p1, p2 in zip(model1.parameters(), model2.parameters()):
    assert p1.shape == p2.shape

In [1]:
import sys
import numpy as np
import pandas as pd
import os
import cv2
import wandb
from datetime import datetime
from tqdm import tqdm
import argparse
import random
import json
import copy


import timm
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import ConcatDataset

from VAL import test

from hc701fed.dataset.WeightedConcatDataset import WeightedConcatDataset
from hc701fed.dataset.dataset_list_transform import (
    APTOS_train,
    EyePACS_train,
    MESSIDOR_2_train,
    MESSIDOR_pairs_train,
    MESSIDOR_Etienne_train,
    MESSIDOR_Brest_train,
)

from hc701fed.dataset.val_dataset_list import (
    APTOS_Val,
    EyePACS_Val,
    MESSIDOR_2_Val,
    MESSIDOR_pairs_Val,
    MESSIDOR_Etienne_Val,
    MESSIDOR_Brest_Val,
)
from hc701fed.dataset.val_dataset_list import (
    APTOS_Test,
    EyePACS_Test,
    MESSIDOR_2_Test,
    MESSIDOR_pairs_Test,
    MESSIDOR_Etienne_Test,
    MESSIDOR_Brest_Test,
)

In [2]:
all_test_dataset = ConcatDataset([APTOS_Test, EyePACS_Test, MESSIDOR_2_Test, MESSIDOR_pairs_Test, MESSIDOR_Etienne_Test, MESSIDOR_Brest_Test])
all_val_dataset = ConcatDataset([APTOS_Val, EyePACS_Val, MESSIDOR_2_Val, MESSIDOR_pairs_Val, MESSIDOR_Etienne_Val, MESSIDOR_Brest_Val])

all_test_loader = DataLoader(all_test_dataset, batch_size=32, shuffle=False, num_workers=4)
all_val_loader = DataLoader(all_val_dataset, batch_size=32, shuffle=False, num_workers=4)

In [3]:
model =  timm.create_model('resnet50', pretrained=False, num_classes=5)
model.load_state_dict(torch.load('/home/xiangjianhou/hc701-fed/checkpoint_simclr_fineturn/centerlized_resnet50_42/20230416_214252/centerlized_resnet50_29_20230416_214252.pth'))

<All keys matched successfully>

In [4]:
val_acc,val_f1 = test(model, 'cuda', all_val_loader)

100%|██████████| 304/304 [00:09<00:00, 32.05it/s]


In [5]:
test_acc,test_f1 = test(model, 'cuda', all_test_loader)

100%|██████████| 309/309 [00:09<00:00, 32.06it/s]


In [6]:
print(val_acc,val_f1,test_acc,test_f1)

0.7954428291576451 0.5549775020862064 0.7881424524484014 0.5429401099336415
